In [111]:
import cd_tools
from cd_tools.osvalkyrie import project_path
import os
import ssl
import chemdataextractor as cder
import cd_lib.chose_para as para

In [113]:
# 设置文件路径，通常用于HTML或XML文件，因为PDF文件格式可能导致段落定位问题
lgrd = cd_tools.lgrd
file_path = os.path.join(project_path(), "DemoDatabase/Publication/document")

# 使用 chemdataextractor 的 Document 类从文件中读取数据
# 'from_file' 方法用于加载文件并创建文档对象
document= cder.Document.from_file(os.path.join(file_path, 'full4.html'))

In [114]:
item = para.cde_paras(document)

if item.sny_sele():
    sny_para_list = item.sny_para_str()
    pot_sny_para_no = 0
    para_number=0
    para_number=len(sny_para_list)
    print(f'We find {para_number} synthesis paragraph(s), which is (are):')
    while pot_sny_para_no < len(sny_para_list):
        print(f'{pot_sny_para_no+1})')
        print(f'{sny_para_list[pot_sny_para_no]})')
        with open(os.path.join(file_path, 'pot_sny_para' + str(pot_sny_para_no) + '.txt'), 'w',
                  encoding='utf-8') as f:
            f.write(sny_para_list[pot_sny_para_no])
        pot_sny_para_no += 1
        print()


We find 2 synthesis paragraph(s), which is (are):
1)

A mixture of Zn(NO3)2·6H2O (0.1 mmol, 0.030 g), H5L (0.04 mmol, 0.018 g), DMF (1.5 mL) and water (0.5 mL) was placed in a screw-capped vial, then the vial was capped and placed in an oven at 105 °C for 72 h. The resulting block crystals were washed with DMF three times to give 1·DMF. The yield was ∼24.0 mg (72.6% based on H5L). Anal. Calcd for C65H74N7O27.5Zn4: C, 47.17; H, 4.51; N, 5.92. Found: C, 47.02; H, 4.78; N, 5.63. IR (cm−1): 3424m, 2965w, 2932w, 2807w, 2492w, 2026w, 1660s, 1628s, 1579s, 1450w, 1435w, 1390s, 1255w, 1163w, 1104m, 1061w, 1020w, 920w, 892w, 853w, 781s, 724s, 665w, 579w, 477w.
)

2)

A mixture of Zn(NO3)2·6H2O (0.1 mmol, 0.030 g), H5L (0.04 mmol, 0.018 g), DMA (1.5 mL) and water (1.0 mL) was placed in a screw-capped vial, then the vial was capped and placed in an oven at 105 °C for 72 h. The resulting block crystals were washed with DMA three times to give 1·DMA. The yield was ∼20.6 mg (60.4% based on H5L). Anal

In [157]:
import re
from cd_lib.pcplib import metal_table, nonmetal_table
def re_metal(text):
# 使用正则表达式查找金属元素
    found_metals = set(re.findall(r'\b(?:' + '|'.join(metal_table) + r')\b', text))

    if not found_metals:
        print("there is no metal")
        result= None
    else:
        print("metal is:"+"".join(found_metals))
        result = ''.join(found_metals)

    return result

for i in range(para_number):
    metal1=re_metal(sny_para_list[i])

metal is:Zn
metal is:Zn


In [117]:
import subprocess
import cd_lib.chetg as ctg
data_para={}
def run_jar(jar_path, args):
    command = ['java', '-jar', jar_path] + args
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    return stdout, stderr

# 示例调用
jar_path = 'E:/bristol graduation project/MOF_Literature_Extraction-main/Literature Extraction/Code/_CommonRedist/chemicalTagger-1.6-SNAPSHOT-jar-with-dependencies-file.jar'

for ind in range(para_number):
    input_file_path = os.path.join(file_path, 'pot_sny_para' + str(ind) + '.txt')
    output_file_path = os.path.join(file_path, 'chemtg' + str(ind)+'.xml')
    # xml_type = ctg.chemtgp(text_loc, chemtg_location= chemtg_location, opt_put='chemtg' + str(ind))
    # 检查文件是否存在，如果不存在则创建它
    if not os.path.exists(output_file_path):
        with open(output_file_path, 'w') as file:
            file.write('<root></root>')  # 创建一个简单的XML文件

    args = [input_file_path, output_file_path]
    stdout, stderr = run_jar(jar_path, args)
    with open(output_file_path, 'r', encoding='utf-8') as file:
        xml_type = file.read()
    print(stdout.decode('utf-8'))
    print(stderr.decode('utf-8'))

    data_para[ind] = {'metal':metal1,'xml': xml_type}


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.



In [118]:
for i in range(len(data_para)):

    item_name = list(data_para.keys())[i]

    # read the result
    item_file = ctg.ctg_xml_par(xml_type)

    # the process to select condition
    csv_cont = [item_name, '']
    print(csv_cont)

[0, '']
[1, '']


In [126]:
from cd_lib import onlystr
from cd_lib.pcplib import metal_table, nonmetal_table

for i in range(len(data_para)):

    item_name = list(data_para.keys())[i]

    # read the result
    item_file = ctg.ctg_xml_par(xml_type)

    # the process to select condition
    csv_cont = [item_name, '']

    def csv_con_bp(chemical_name):
        bp_man = ['calc', 'TGA', 'Teflon', 'plate']
        breakpoint_list = metal_table + nonmetal_table + bp_man
        for i in chemical_name.strip().split('-'):
            if i in breakpoint_list:
                return True
        else:
            return False

    try:
        tt_ele_cla = ctg.tt_classifer(item_file.ope_list())
        tt_re = ctg.ht_ident(tt_ele_cla)
    except ctg.NoTemTimeError as e:
        lgrd.warning('{}_{}:{}'.format(item_name, repr(e), str(e)))
        csv_cont[1] = csv_cont[1] + 'T'
        tt_re = {'temp': '', 'temp_u': '', 'time': '', 'time_u': ''}
    except (TypeError, ValueError) as e:
        lgrd.warn('{}_{}:{}'.format(item_name, repr(e), str(e)))
        csv_cont[1] = csv_cont[1] + 'T'
        tt_re = {'temp': '', 'temp_u': '', 'time': '', 'time_u': ''}

    for i in ['temp', 'temp_u', 'time', 'time_u']:
        csv_cont.append(onlystr(tt_re[i]))

    try:
        item_yield = ctg.yield_out(item_file.yield_list())
    except Exception as err:
        lgrd.warn('{}_{}:{}'.format(item_name, repr(err), str(err)))
        item_yield = None
    finally:
        if item_yield is None:
            csv_cont.append('')
        else:
            csv_cont.append(item_yield)
    # 到这, 已经前面的坑全部占了
    # 方法结束

    item_metal_data = data_para[item_name].get('metal')
    if item_metal_data is None:
        lgrd.error(f"Metal data for {item_name} is missing.")
        continue

    item_metal = item_metal_data.split(',')
    item_metal = set([x.strip() for x in item_metal])

    metal_cont = []
    chem_cont = []

    try:
        item_chemical_list_row = ctg.cc_i(item_file.molecular_list())
        item_chemical_list = ctg.mixture_iden_no_qua(item_chemical_list_row)
        item_chemical_out = ctg.cc_out_no_qua(item_chemical_list)
        item_chemical_out = ctg.cc_table_no_qua(item_chemical_out)
    except (ctg.ChemicalListError, ctg.NullChemicalNameError, ValueError, AssertionError, TypeError) as e:
        csv_cont[1] = csv_cont[1] + 'C'
        lgrd.warning('{}_{}:{}'.format(item_name, repr(e), str(e)))

        item_chemical_out = []

    if len(item_chemical_out) > 2:
        for i in item_chemical_out:
            if len(i) == 3 and i['name'] != "":
                table = ['name', 'cid', 'chem_role']
                chemicals_info = [i[x] for x in table]
            else:
                csv_cont[1] = csv_cont[1] + 'B'
                chemicals_info = []

            if len(chemicals_info) == 3:
                if csv_con_bp(chemicals_info[0]):
                    break
                else:
                    if chemicals_info[-1].strip() != "" and (chemicals_info[-1].strip() in item_metal) and (len(metal_cont) < 15):
                        metal_cont = metal_cont + chemicals_info
                    else:
                        chem_cont = chem_cont + chemicals_info
            else:
                csv_cont[1] = csv_cont[1] + 'C'
    else:
        csv_cont[1] = csv_cont[1] + 'S'
    lgrd.info(f'{item_name}:{metal_cont}\t{chem_cont}')

    if len(metal_cont) == 0:
        csv_cont[1] = csv_cont[1] + 'M'

    while len(metal_cont) < 15:
        metal_cont = metal_cont + [""]

    if len(chem_cont) > 30:
        chem_cont = chem_cont[:30]
        csv_cont[1] = csv_cont[1] + 'L'
    csv_cont = csv_cont + metal_cont + chem_cont
    csv_cont = [onlystr(x) for x in csv_cont]
    data_para[item_name]['metal_extracted'] = metal_cont
    data_para[item_name]['condition_extracted'] = csv_cont
    data_para[item_name]['chemical_extracted'] = chem_cont

[2024-07-23 10:24:50,707] - [C&C] - [20153740.py file line:93] - INFO: 0:['Zn(NO3)2·6H2O', None, 'Zn', 'C64H85N6O31.5Zn4', None, 'Zn']	['H5L', 91820518, '', 'DMA', 31374, 'Sol', 'water', 962, 'Sol', '1·DMA', None, '', 'H5L', 91820518, '']
zzy1 ['Zn(NO3)2·6H2O', None, 'Zn', 'C64H85N6O31.5Zn4', None, 'Zn', '', '', '', '', '', '', '', '', '']
zzy2 ['0', '', '105.0', '°C', '72', 'h', '', 'Zn(NO3)2·6H2O', '', 'Zn', 'C64H85N6O31.5Zn4', '', 'Zn', '', '', '', '', '', '', '', '', '', 'H5L', '91820518', '', 'DMA', '31374', 'Sol', 'water', '962', 'Sol', '1·DMA', '', '', 'H5L', '91820518', '']
zzy3 ['H5L', 91820518, '', 'DMA', 31374, 'Sol', 'water', 962, 'Sol', '1·DMA', None, '', 'H5L', 91820518, '']
[2024-07-23 10:24:57,935] - [C&C] - [20153740.py file line:93] - INFO: 1:['Zn(NO3)2·6H2O', None, 'Zn', 'C64H85N6O31.5Zn4', None, 'Zn']	['H5L', 91820518, '', 'DMA', 31374, 'Sol', 'water', 962, 'Sol', '1·DMA', None, '', 'H5L', 91820518, '']
zzy1 ['Zn(NO3)2·6H2O', None, 'Zn', 'C64H85N6O31.5Zn4', None, 'Z

In [127]:
for i in range(len(data_para)):

    item_name = list(data_para.keys())[i]

    # if not all the Open Metal Sites extracted in the literature, we skip it.
    extracted_metal_list = set(data_para[item_name]['metal_extracted'][2::3])
    print(extracted_metal_list)

{'', 'Zn'}
{'', 'Zn'}


In [154]:
import pandas as pd
pd_dict = []
for i in range(len(data_para)):

    item_name = list(data_para.keys())[i]

    # if not all the Open Metal Sites extracted in the literature, we skip it.
    extracted_metal_list = set(data_para[item_name]['metal_extracted'][2::3])
    for metal in data_para[item_name]['metal']:
        if metal not in extracted_metal_list:
            continue
    temp_dic= {}
    temp_dic['filename'] = item_name
    temp_dic['metal']=metal1
    temp_dic['temperature'] =data_para[item_name]['condition_extracted'][2]
    temp_dic['temperature_unit'] =data_para[item_name]['condition_extracted'][3]
    temp_dic['time']=data_para[item_name]['condition_extracted'][4]
    temp_dic['time_unit']=data_para[item_name]['condition_extracted'][5]
    solves_name = []
    solves_CID = []
    additves_name = []
    additves_CID = []
    ind = 1
    while ind < len(data_para[item_name]['chemical_extracted']):
        if data_para[item_name]['chemical_extracted'][ind+1] == "Sol":
            solves_name.append(data_para[item_name]['chemical_extracted'][ind-1])
            solves_CID.append(data_para[item_name]['chemical_extracted'][ind])
        elif data_para[item_name]['chemical_extracted'][ind+1] == "Addi":
            additves_name.append(data_para[item_name]['chemical_extracted'][ind]-1)
            additves_CID.append(data_para[item_name]['chemical_extracted'][ind])
        ind += 3

    for p in range(len(solves_name)):
        temp_dic[f'solvent name{p+1}'] = solves_name[p]
        temp_dic[f'solvent CID{p+1}'] = solves_CID[p]

    for p in range(len(additves_name)):
        temp_dic[f'additive name{p+1}'] = additves_name[p]
        temp_dic[f'additive CID{p+1}'] = additves_CID[p]

    pd_dict.append(temp_dic)
df = pd.DataFrame(pd_dict)

In [155]:
df

,filename,metal,temperature,temperature_unit,time,time_unit,solvent name1,solvent CID1,solvent name2,solvent CID2
0,0,Zn,105.0,°C,72,h,DMA,31374,water,962
1,1,Zn,105.0,°C,72,h,DMA,31374,water,962
